In [267]:
# P(A|B) = P(B|A) * P(A) / P(B)
p_A = 0
p_not_A = 0

train_positive = {}
train_negative = {}

positive_total = 0
negative_total = 0

In [268]:
def train(data):
    global p_A, p_not_A
    num_spam = 0
    total = 0
    for email in data:
        if email['label'] == 'SPAM':
            num_spam += 1
        total += 1
        process_email(email['body'], email['label'])
    p_A = num_spam / float(total)
    p_not_A = (total - num_spam) / float(total)

# count the words in a specific email
def process_email(body, label):
    global positive_total, negative_total, train_positive, train_negative
    for word in body:
        if label == 'SPAM':
            train_positive[word] = train_positive.get(word, 0) + 1
            positive_total += 1
        else:
            train_negative[word] = train_negative.get(word, 0) + 1
            negative_total += 1

# Gives the conditional probability p(B_i | A_x)
def conditional_word(word, spam):
    global positive_total, negative_total, train_positive, train_negative
    if positive_total == 0 or negative_total == 0:
        return 0
    if spam:
        return train_positive.get(word, 0) / float(positive_total)
    return train_negative.get(word, 0) / float(negative_total)

# To get P(B | A_x) for an entire email, we simply take the product of the P(B_i | A_x) value for every word i in the email
def conditional_email(body, spam):
    result = 1.0
    for word in body:
        result *= conditional_word(word, spam)
    return result

# Predict whether an email is spam or not
def predict(X):
    is_spam = p_A * conditional_email(X, True)
    not_spam = p_not_A * conditional_email(X, False)
    return is_spam > not_spam

In [269]:
data = [
    { 'body': ['spam', 'bad'], 'label': 'SPAM' },
    { 'body': ['hello', 'world', 'buy'], 'label': 'NOT_SPAM' },
]

train(data)

print(predict(['spam', 'bad']))
print(predict(['world', 'hello', 'hello']))
print(predict(['world', 'buy', 'spam']))

True
False
False
